In [3]:
using Pkg
Pkg.add("Flux")
using Flux.Tracker

  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `/opt/julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.1/Manifest.toml`
 [no changes]


# Gradientes

### Automatic differentiation

In [4]:
f(x) = 3x^2 + 2x + 1;


In [5]:
df(x) = Tracker.gradient(f, x; nest = true)[1];

In [6]:
df(2)

14.0 (tracked)

In [7]:
d2f(x) = Tracker.gradient(df, x; nest = true)[1];

In [8]:
Tracker.gradient(f, 2; nest = true)

(14.0 (tracked),)

Podemos sacar el gradiente con respecto a múltiples parámetros

In [9]:
f(W, b, x) = W * x + b;
Tracker.gradient(f, 2, 3, 4)

(4.0 (tracked), 1.0 (tracked), 2.0 (tracked))

Podemos tener cientos de parámetros!
Podemos tratar a nuestros parámetros como 'params', así los convertimos en números duales

In [10]:
using Flux

In [ ]:
W = param(2) 
b = param(3)
f(x) = W * x + b;

gradient es una zero-argument function. No necesita los argumentos porque params indica qué diferenciar

In [ ]:
grads = Tracker.gradient(() -> f(4), params(W, b));

### Resultado: 
La derivada de f(W, b), el gradiente que obtenemos es un vector con los componentes grads[W] y grads[b]

In [ ]:
grads[W]

In [11]:
grads[b]

1.0 (tracked)

## Modelo simple de regresión lineal

In [ ]:
using Flux.Tracker
W = rand(2, 5)
b = rand(2)

predict(x) = W*x .+ b

function loss(x, y)
  ŷ = predict(x)
  sum((y .- ŷ).^2)
end

x, y = rand(5), rand(2) 
loss(x, y) 

 * Queremos minimizar la loss function.
 * Tomamos los gradientes de W y b con respecto la función de loss
 * Aplicamos gradient descent 

In [18]:
W = param(W)
b = param(b)

gs = Tracker.gradient(() -> loss(x, y), params(W, b))

Grads(...)


* Teniendo los gradientes podemos ir actualizando W para entrenar el modelo
* update!(W, Δ) aplica W = W + Δ

In [ ]:
using Flux.Tracker: update!

Δ = gs[W]
update!(W, -0.1Δ)

loss(x, y) 

Usando estas herramientas podemos llevar a cabo, en principio, modelos de Deep Learning

# Capas

### Redes Neuronales
* Básicamente están construídas por muchas que pasan su output por una función de activación.
* Vamos a usar la función de activación sigmoid (σ) que es muy común.
* Después el output se pasa a la siguiente capa

Forma naive de crear una red

In [19]:
W1 = param(rand(3, 5))
b1 = param(rand(3))
layer1(x) = W1 * x .+ b1

W2 = param(rand(2, 3))
b2 = param(rand(2))
layer2(x) = W2 * x .+ b2

model(x) = layer2(σ.(layer1(x)))

model(rand(5))

Tracked 2-element Array{Float64,1}:
 2.392631575799201
 2.734965056940612

Creamos una función para crear capas lineales

In [14]:
function linear(in, out)
  W = param(randn(out, in))
  b = param(randn(out))
  x -> W * x .+ b
end

linear1 = linear(5, 3) 
linear2 = linear(3, 2)

model(x) = linear2(σ.(linear1(x)))

model(rand(5)) 

Tracked 2-element Array{Float64,1}:
  1.077676442887153  
 -0.07162517761343418

* Podemos tener una lista de capas con la función de output final a aplicar
* foldl funciona como un reduce
* va pasando al vector x transformandolo en cada capa de la lista

In [15]:

layers = [Dense(10, 5, σ), Dense(5, 2), softmax]

model(x) = foldl((x, m) -> m(x), layers, init = x)

model(rand(10)) 

Tracked 2-element Array{Float32,1}:
 0.392105f0
 0.607895f0

Flux nos facilita el contrsuctor Chain para no tener que hacer ese reduce nosotros mismos

In [16]:
model2 = Chain(
  Dense(10, 5, σ),
  Dense(5, 2),
  softmax)

model2(rand(10))

Tracked 2-element Array{Float32,1}:
 0.46175814f0
 0.53824186f0

También podemos pensar a Chain como una composición de funciones!

In [17]:
m = Dense(5, 2) ∘ Dense(10, 5, σ)

m(rand(10))

Tracked 2-element Array{Float32,1}:
 -0.24105902f0
 -0.5822575f0 

Podemos usar Chain también para encadenar cualquier tipo de funciones de Julia

In [20]:
m = Chain(x -> x^2, x -> x+1)

m(5)

26